In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from preprocessing import parse_aug_fn, parse_fn

In [3]:
# 將train Data重新分成9:1等分，分別分給train data, valid data
train_split, valid_split = ['train[:90%]', 'train[90%:]']
# 取得訓練數據，並順便讀取data的資訊
train_data, info = tfds.load("cifar10", split=train_split, with_info=True)
# 取得驗證數據
valid_data = tfds.load("cifar10", split=valid_split)
# 取得測試數據
test_data = tfds.load("cifar10", split=tfds.Split.TEST)

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE  # 自動調整模式
batch_size = 64  # 批次大小
train_num = int(info.splits['train'].num_examples / 10) * 9  # 訓練資料數量

train_data = train_data.shuffle(train_num)  # 打散資料集
# 載入預處理「 parse_aug_fn」function，cpu數量為自動調整模式
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [5]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(inputs)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(256, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)
# 建立網路模型(將輸入到輸出所有經過的網路層連接起來)
model_1 = keras.Model(inputs, outputs, name='model-1')
model_1.summary()

Model: "model-1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         295040    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          7379

In [6]:
# 儲存訓練記錄檔
logs_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dir='lab6-logs')

# 紀錄每一個batch的Loss值變化
model_dirs = logs_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)
save_model = tf.keras.callbacks.ModelCheckpoint(model_dirs + '/save.h5', 
                                                monitor='val_catrgorical_accuracy', 
                                                mode='max')

In [7]:
# 設定訓練使用的優化器、損失函數和指標函數
model_1.compile(keras.optimizers.Adam(), 
                loss=keras.losses.CategoricalCrossentropy(from_logits=True), 
                metrics=[keras.metrics.CategoricalAccuracy()])


In [8]:
# 訓練網路模型
model_1.fit(train_data,
            epochs=100, 
            validation_data=valid_data,
            callbacks=[model_cbk, save_model])

Epoch 1/100
704/704 [==============================] - 33s 47ms/step - loss: 2.0936 - categorical_accuracy: 0.2043 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 19s 27ms/step - loss: 1.9173 - categorical_accuracy: 0.2923 - val_loss: 1.6245 - val_categorical_accuracy: 0.4190
Epoch 3/100
704/704 [==============================] - 20s 28ms/step - loss: 1.7746 - categorical_accuracy: 0.3548 - val_loss: 1.4580 - val_categorical_accuracy: 0.4704
Epoch 4/100
704/704 [==============================] - 19s 27ms/step - loss: 1.6751 - categorical_accuracy: 0.3936 - val_loss: 1.3459 - val_categorical_accuracy: 0.5178
Epoch 5/100
704/704 [==============================] - 19s 27ms/step - loss: 1.6161 - categorical_accuracy: 0.4177 - val_loss: 1.3117 - val_categorical_accuracy: 0.5266
Epoch 6/100
704/704 [==============================] - 19s 27ms/step - loss: 1.5568 - categorical_accuracy: 0.4438 - val_loss: 1.2263 - val_categor

704/704 [==============================] - 19s 27ms/step - loss: 0.9159 - categorical_accuracy: 0.6896 - val_loss: 0.7221 - val_categorical_accuracy: 0.7522
Epoch 50/100
704/704 [==============================] - 19s 27ms/step - loss: 0.9114 - categorical_accuracy: 0.6932 - val_loss: 0.7488 - val_categorical_accuracy: 0.7500
Epoch 51/100
704/704 [==============================] - 19s 27ms/step - loss: 0.9088 - categorical_accuracy: 0.6923 - val_loss: 0.6825 - val_categorical_accuracy: 0.7738
Epoch 52/100
704/704 [==============================] - 19s 27ms/step - loss: 0.8997 - categorical_accuracy: 0.6950 - val_loss: 0.7158 - val_categorical_accuracy: 0.7566
Epoch 53/100
704/704 [==============================] - 19s 27ms/step - loss: 0.9004 - categorical_accuracy: 0.6973 - val_loss: 0.6969 - val_categorical_accuracy: 0.7668
Epoch 54/100
704/704 [==============================] - 19s 27ms/step - loss: 0.8939 - categorical_accuracy: 0.6990 - val_loss: 0.6738 - val_categorical_accuracy: 

704/704 [==============================] - 21s 29ms/step - loss: 0.7869 - categorical_accuracy: 0.7392 - val_loss: 0.6346 - val_categorical_accuracy: 0.7956
Epoch 98/100
704/704 [==============================] - 19s 27ms/step - loss: 0.7829 - categorical_accuracy: 0.7388 - val_loss: 0.6436 - val_categorical_accuracy: 0.7918
Epoch 99/100
704/704 [==============================] - 19s 27ms/step - loss: 0.7959 - categorical_accuracy: 0.7375 - val_loss: 0.6205 - val_categorical_accuracy: 0.7968
Epoch 100/100
704/704 [==============================] - 19s 27ms/step - loss: 0.7917 - categorical_accuracy: 0.7363 - val_loss: 0.6523 - val_categorical_accuracy: 0.7862


In [30]:
class CustomConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding="VALID", **kwargs):
        super(CustomConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = (1, *strides, 1)
        self.padding = padding

    def build(self, input_shape):
        kernel_h, kernel_w = self.kernel_size
        input_dim = input_shape[-1]
        # 創建卷積的權重值(weights)
        self.w = self.add_weight(name='kernel', 
                                 shape=(kernel_h, kernel_w, input_dim, self.filters),
                                 initializer='glorot_uniform',  # 設定初始化方法
                                 trainable=True)  # 設定這個權重是否能夠訓練(更動)
        # 創建卷積的偏差值(bias)
        self.b = self.add_weight(name='bias', 
                                 shape=(self.filters,),
                                 initializer='zeros',  # 設定初始化方法
                                 trainable=True)  # 設定這個權重是否能夠訓練(更動)

    def call(self, inputs):
        x = tf.nn.conv2d(inputs, self.w, self.strides, padding=self.padding) # 卷積運算
        x = tf.nn.bias_add(x, self.b)  # 加上偏差值
        x = tf.nn.relu(x)  # 激活函數
        return x

In [31]:
inputs = keras.Input(shape=(32, 32, 3))
x = CustomConv2D(64, (3, 3))(inputs)
x = layers.MaxPool2D()(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(256, (3, 3))(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(64, (3, 3))(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)
# 建立網路模型(將輸入到輸出所有經過的網路層連接起來)
model_2 = keras.Model(inputs, outputs, name='model-2')
model_2.summary()

Model: "model-2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
custom_conv2d_10 (CustomConv (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
custom_conv2d_11 (CustomConv (None, 13, 13, 128)       73856     
_________________________________________________________________
custom_conv2d_12 (CustomConv (None, 11, 11, 256)       295168    
_________________________________________________________________
custom_conv2d_13 (CustomConv (None, 9, 9, 128)         295040    
_________________________________________________________________
custom_conv2d_14 (CustomConv (None, 7, 7, 64)          7379

In [32]:
class SaveModel(tf.keras.callbacks.Callback):
    def __init__(self, weights_file, monitor='loss', mode='min', save_weights_only=False):
        super(SaveModel, self).__init__()
        self.weights_file = weights_file
        self.monitor = monitor
        self.mode = mode
        self.save_weights_only = save_weights_only
        if mode == 'min':
            self.best = np.Inf
        else:
            self.best = -np.Inf
        
    def save_model(self):
        if self.save_weights_only:
            self.model.save_weights(self.weights_file)
        else:
            self.model.save(self.weights_file)

    def on_epoch_end(self, epoch, logs=None):
        monitor_value = logs.get(self.monitor)
        if self.mode == 'min' and monitor_value < self.best:
            self.save_model()
            self.best = monitor_value
        elif self.mode == 'max' and monitor_value > self.best:
            self.save_model()
            self.best = monitor_value

In [33]:
# 儲存訓練記錄檔
logs_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dir='lab6-logs')

# 紀錄每一個batch的Loss值變化
model_dirs = logs_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)
custom_save_model = SaveModel(model_dirs + '/custom_save.h5', 
                              monitor='val_custom_catrgorical_accuracy', 
                              mode='max', 
                              save_weights_only=True)

In [34]:
def custom_categorical_crossentropy(y_true, y_pred):
    # x = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_pred), reduction_indices=[1]))
    x = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return x

In [35]:
class CustomCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_catrgorical_accuracy', **kwargs):
        super(CustomCategoricalAccuracy, self).__init__(name=name, **kwargs)
        # 記錄正確預測的數量
        self.correct = self.add_weight('correct_numbers', initializer='zeros')
        # 記錄全部資料的量數
        self.total = self.add_weight('total_numbers', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # 輸入答案為One-hot編碼，所以取最大的數值為答案
        y_true = tf.argmax(y_true, axis=-1)
        # 取預測輸出最大的數值為預測結果
        y_pred = tf.argmax(y_pred, axis=-1)
        # 比較預測結果是否正確，正確會返回True(正確)，錯誤會返回False(錯誤)
        values = tf.equal(y_true, y_pred)
        # 轉為浮點數True(正確)=1.0，False(錯誤)=0.0
        values = tf.cast(values, tf.float32)
        # 將values所有數值相加就會等於正確預測的總數
        values_sum = tf.reduce_sum(values)
        # 計算這個Batch的資料數量
        num_values = tf.cast(tf.size(values), tf.float32)
        self.correct.assign_add(values_sum)  # 更新正確預測的總數
        self.total.assign_add(num_values)  # 更新資料量的總數

    def result(self):
        # 計算準確率
        return tf.math.divide_no_nan(self.correct, self.total)

    def reset_states(self):
        # 每一次Epoch結束後會重新初始化變數
        self.correct.assign(0.)
        self.total.assign(0.)

In [36]:
model_2.compile(keras.optimizers.Adam(),
                loss=custom_categorical_crossentropy,
                metrics=[CustomCategoricalAccuracy()])

In [37]:
model_2.fit(train_data,
            epochs=100,
            validation_data=valid_data,
            callbacks=[model_cbk, custom_save_model])

Epoch 1/100
704/704 [==============================] - 21s 30ms/step - loss: 2.1002 - custom_catrgorical_accuracy: 0.1942 - val_loss: 0.0000e+00 - val_custom_catrgorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 19s 27ms/step - loss: 1.9397 - custom_catrgorical_accuracy: 0.2822 - val_loss: 1.6238 - val_custom_catrgorical_accuracy: 0.4150
Epoch 3/100
704/704 [==============================] - 19s 27ms/step - loss: 1.8009 - custom_catrgorical_accuracy: 0.3447 - val_loss: 1.5402 - val_custom_catrgorical_accuracy: 0.4422
Epoch 4/100
704/704 [==============================] - 19s 27ms/step - loss: 1.6965 - custom_catrgorical_accuracy: 0.3864 - val_loss: 1.4064 - val_custom_catrgorical_accuracy: 0.5030
Epoch 5/100
704/704 [==============================] - 19s 27ms/step - loss: 1.6202 - custom_catrgorical_accuracy: 0.4178 - val_loss: 1.3224 - val_custom_catrgorical_accuracy: 0.5418
Epoch 6/100
704/704 [==============================] - 19s 27ms/step - loss: 

704/704 [==============================] - 19s 27ms/step - loss: 0.9869 - custom_catrgorical_accuracy: 0.6644 - val_loss: 0.7345 - val_custom_catrgorical_accuracy: 0.7496
Epoch 46/100
704/704 [==============================] - 19s 27ms/step - loss: 0.9811 - custom_catrgorical_accuracy: 0.6655 - val_loss: 0.8037 - val_custom_catrgorical_accuracy: 0.7306
Epoch 47/100
704/704 [==============================] - 19s 27ms/step - loss: 0.9812 - custom_catrgorical_accuracy: 0.6686 - val_loss: 0.7367 - val_custom_catrgorical_accuracy: 0.7498
Epoch 48/100
704/704 [==============================] - 19s 27ms/step - loss: 0.9684 - custom_catrgorical_accuracy: 0.6693 - val_loss: 0.7467 - val_custom_catrgorical_accuracy: 0.7458
Epoch 49/100
704/704 [==============================] - 19s 27ms/step - loss: 0.9682 - custom_catrgorical_accuracy: 0.6759 - val_loss: 0.7134 - val_custom_catrgorical_accuracy: 0.7572
Epoch 50/100
704/704 [==============================] - 19s 27ms/step - loss: 0.9589 - custom

704/704 [==============================] - 19s 26ms/step - loss: 0.8384 - custom_catrgorical_accuracy: 0.7211 - val_loss: 0.6494 - val_custom_catrgorical_accuracy: 0.7830
Epoch 90/100
704/704 [==============================] - 19s 26ms/step - loss: 0.8393 - custom_catrgorical_accuracy: 0.7235 - val_loss: 0.6827 - val_custom_catrgorical_accuracy: 0.7642
Epoch 91/100
704/704 [==============================] - 19s 26ms/step - loss: 0.8346 - custom_catrgorical_accuracy: 0.7219 - val_loss: 0.6807 - val_custom_catrgorical_accuracy: 0.7710
Epoch 92/100
704/704 [==============================] - 19s 27ms/step - loss: 0.8455 - custom_catrgorical_accuracy: 0.7198 - val_loss: 0.6447 - val_custom_catrgorical_accuracy: 0.7850
Epoch 93/100
704/704 [==============================] - 19s 27ms/step - loss: 0.8389 - custom_catrgorical_accuracy: 0.7196 - val_loss: 0.6548 - val_custom_catrgorical_accuracy: 0.7852
Epoch 94/100
704/704 [==============================] - 19s 27ms/step - loss: 0.8247 - custom

In [38]:
model_1.load_weights(model_dirs+'/save.h5')
model_2.load_weights(model_dirs+'/custom_save.h5')

In [39]:
loss_1, acc_1 = model_1.evaluate(test_data)
loss_2, acc_2 = model_2.evaluate(test_data)
loss = [loss_1, loss_2]
acc = [acc_1, acc_2]
dict = {"Loss":loss, "Accuracy": acc}
pd.DataFrame(dict)

157/157 [==============================] - 2s 12ms/step - loss: 0.6416 - custom_catrgorical_accuracy: 0.7916


,Loss,Accuracy
0,0.652324,0.7890
1,0.641553,0.7916
